In [1]:
# Importing all the required libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
#Importing the train data set
df_train = pd.read_csv('train.csv',encoding = "ISO-8859-1")

In [3]:
df_train.head()

,Source,Host,Link,Date(ET),Time(ET),time(GMT),Title,TRANS_CONV_TEXT,Patient_Tag
0,FORUMS,cafepharma.com,http://cafepharma.com/boards/threads/epstein.5...,6/15/2016,13:58:00,6/15/2016 23:28,Epstein,I don't disagree with you in principle. I'm ju...,0
1,FORUMS,www.patient.co.uk,http://www.patient.co.uk/forums/discuss/enlarg...,5/7/2016,0.820833333,42498.21667,Enlarged Heart.Thread Enlarged Heart,I am always dizzy I get dizzy standing up so I...,1
2,BLOG,http://abcnewsradioonline.com/entertainment-news,http://abcnewsradioonline.com/entertainment-ne...,4/14/2016,15:00:38,4/15/2016 0:30,Queen Latifah Joins American Heart Association...,Axelle/Bauer-Griffin/FilmMagic(NEW YORK) -- Qu...,0
3,FORUMS,www.cancer-forums.net,http://www.cancer-forums.net/viewtopic.php?f=1...,6/18/2016,20:46:00,6/19/2016 6:16,Bulaemia,I am 17 and I have been throwing up for about ...,1
4,FORUMS,www.diyaudio.com,http://www.diyaudio.com/forums/lounge/292252-d...,6/15/2016,3:26:00,6/15/2016 12:56,DIY Silver interconnects and RCAs???,Quote: Originally Posted by Boyan Silyavski Wa...,0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 9 columns):
Source             1157 non-null object
Host               1098 non-null object
Link               1157 non-null object
Date(ET)           1157 non-null object
Time(ET)           1157 non-null object
time(GMT)          996 non-null object
Title              941 non-null object
TRANS_CONV_TEXT    1156 non-null object
Patient_Tag        1157 non-null int64
dtypes: int64(1), object(8)
memory usage: 81.4+ KB


In [5]:
df_train.nunique()

Source                5
Host                615
Link               1152
Date(ET)            167
Time(ET)            962
time(GMT)           967
Title               878
TRANS_CONV_TEXT    1156
Patient_Tag           2
dtype: int64

In [6]:
df_train.drop(['Date(ET)','Time(ET)','time(GMT)'],axis=1,inplace=True)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 6 columns):
Source             1157 non-null object
Host               1098 non-null object
Link               1157 non-null object
Title              941 non-null object
TRANS_CONV_TEXT    1156 non-null object
Patient_Tag        1157 non-null int64
dtypes: int64(1), object(5)
memory usage: 54.3+ KB


In [8]:
df_train.drop('Host',axis=1, inplace=True)

In [9]:
from urllib.parse import urlparse

In [10]:
def get_url(x):
    y = urlparse(x)
    return(y.netloc)

In [11]:
df_train['Host'] = df_train['Link'].apply(lambda x: get_url(x))

In [12]:
def get_title(x):
    y = urlparse(x)
    y = y.path
    y = y.split(sep='/')
    return(y[-1])

In [13]:
lst = df_train[df_train['Title'].isna()].index.tolist()
#lst

In [ ]:
for i in lst:
    df_train['Title'].iloc[i] = get_title(df_train['Link'].iloc[i])

C:\Users\No_ob0dy\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
df_train.info()

In [ ]:
df_train.dropna(axis=0,inplace=True)
df_train.reset_index(drop=True,inplace=True)

In [ ]:
train1 = pd.DataFrame()

In [ ]:
train1['Text'] = df_train[['Host','Title','TRANS_CONV_TEXT']].apply(lambda x: ' '.join(x),axis=1)

In [ ]:
train = pd.concat([train1,df_train[['Source']]],axis=1)
del train1

In [ ]:
train.head()

In [ ]:
target = df_train[['Patient_Tag']]
del df_train

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
stopwords.extend(['www','co','com','org','net','uk','blogspot','info','de'])

In [ ]:
wordnet = WordNetLemmatizer()
def pre_process(msg,corpus):
    msg = msg.lower()
    msg = re.sub('[^a-zA-Z]',' ', msg)
    msg = nltk.word_tokenize(msg)
    msg = [wordnet.lemmatize(word) for word in msg if word not in stopwords]
    msg = ' '.join(msg)
    corpus.append(msg)

In [ ]:
corpus = []
train['Text'].apply(lambda x : pre_process(x,corpus))
corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(corpus)
x = tfidf.transform(corpus).toarray()

In [ ]:
x1 = pd.DataFrame(x)
x1.head()

In [ ]:
train_dums = pd.get_dummies(train[['Source']],drop_first=True)
train_dums.head()

In [ ]:
del train
train = pd.concat([train_dums,x1],axis=1)
del x1, train_dums, x

In [ ]:
train.head()

In [ ]:
target['Patient_Tag'].value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 1)
train, target = sm.fit_sample(train, target)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train,target,test_size=.30,random_state=1)
print("x_train shape:",x_train.shape)
print("y_train shape:",y_train.shape)
print("x_test shape:",x_test.shape)
print("y_test shape:",y_test.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mNB = MultinomialNB().fit(x_train,y_train)

In [ ]:
y_pred = mNB.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score
#confusion matrix
cnf = confusion_matrix(y_test,y_pred)
cnf

In [ ]:
print("accuracy score:",accuracy_score(y_test,y_pred))
print("precision score:",precision_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()
grid_params = {'n_estimators': [200,500,700,1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']}
rfc1 = GridSearchCV(rfc,grid_params,cv = 10)
rfc1.fit(x_train,y_train)

In [ ]:
rfc1.best_params_

In [ ]:
y_pred2 = rfc1.predict(x_test)

In [ ]:
#confusion matrix
cnf2 = confusion_matrix(y_test,y_pred2)
cnf2

In [ ]:
print("accuracy score:",accuracy_score(y_test,y_pred2))
print("precision score:",precision_score(y_test,y_pred2))

In [ ]:
print(classification_report(y_test,y_pred2))

##### Test data

In [ ]:
df_test = pd.read_csv('test.csv',encoding = "ISO-8859-1")

In [ ]:
df_test.head()

In [ ]:
df_test.isna().sum()

In [ ]:
df_test.drop(['Host','Date(ET)','Time(ET)','time(GMT)','Unnamed: 9'],axis=1,inplace=True)
df_test.info()

In [ ]:
df_test['Host'] = df_test['Link'].apply(lambda x: get_url(x))

In [ ]:
lst1 = df_test[df_test['Title'].isna()].index.tolist()

In [ ]:
for i in lst1:
    df_test['Title'].iloc[i] = get_title(df_test['Link'].iloc[i])

In [ ]:
df_test.isna().sum()

In [ ]:
test1 = pd.DataFrame()
test1['Text'] = df_test[['Host','Title','TRANS_CONV_TEXT']].apply(lambda x: ' '.join(x),axis=1)

In [ ]:
corpus1 = []
test1['Text'].apply(lambda x : pre_process(x,corpus1))
corpus1

In [ ]:
#tfidf1 = TfidfVectorizer(max_features=13000)
x2 = tfidf.transform(corpus1).toarray()

In [ ]:
x3 = pd.DataFrame(x2)
x3.head()

In [ ]:
x3. info()

In [ ]:
test_dums = pd.get_dummies(df_test[['Source']],drop_first=True)
test_dums.head()

In [ ]:
test = pd.concat([test_dums,x3],axis=1)

In [ ]:
test.info()

In [ ]:
del test1
del test_dums,x3,x2

In [ ]:
test_pred = rfc1.predict(test)

In [ ]:
test_pred

In [ ]:
preds = pd.DataFrame(test_pred,columns=['Patient_Tag'])

In [ ]:
preds.reset_index(inplace=True)

In [ ]:
preds.rename(columns={'index':'Index'},inplace=True)

In [ ]:
preds['Index'] = preds['Index']+1
preds.head()

In [ ]:
preds.to_csv('Submission2.csv',index=False)